**Pré-Tutorial**


In [ ]:
!pip install yfinance
!pip install --upgrade matplotlib

In [ ]:
plt.rcParams['figure.figsize'] = [6,4]
plt.rcParams['figure.dpi'] = 100

**Tutorial**

In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
import math

In [ ]:
data = yf.download('^GSPC','2016-01-01','2020-10-01')
data.head()

In [ ]:
def crossover_sinal(data):
  #Criar Sinais de compra e venda
  data['SMA15'] = data['Close'].rolling(15).mean()
  data['SMA50'] = data['Close'].rolling(50).mean()

  data['Anter'] = data['SMA15'].shift(1) - data['SMA50'].shift(1)
  data['Atual'] = data['SMA15'] - data['SMA50']

  data.loc[(data['Anter']<0)&(data['Atual']>0),'Compra'] = data['Close']
  data.loc[(data['Anter']>0)&(data['Atual']<0),'Venda'] = data['Close']

  columns = ['Close','SMA15','SMA50','Compra','Venda']

  return data[columns]

In [ ]:
output_df = crossover_sinal(data)
output_df.tail()

In [ ]:
filtered_data = output_df.loc[(output_df.index<'2020-12-01') & (output_df.index>'2019-01-01')]

In [ ]:
fig, ax = plt.subplots()

#Adicionar Informação
ax.plot(filtered_data.index,filtered_data['Close'],label='Fecho',alpha=0.5)
ax.plot(filtered_data.index,filtered_data['SMA15'],label='MMS15',color='orange')
ax.plot(filtered_data.index,filtered_data['SMA50'],label='MMS50',color='brown')

#Adicionar Venda e Compra
ax.scatter(filtered_data.index,filtered_data['Compra'],label='Compra',marker='^',color='green')
ax.scatter(filtered_data.index,filtered_data['Venda'],label='Venda',marker='v',color='red')

#formatar
fig.autofmt_xdate()
ax.legend()

#Labels adicionais
ax.yaxis.set_major_formatter('${x:}')
ax.set_title('Estratégia Médias Móveis')
ax.set_xlabel('Data')